In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Load dataset
dataset_path = r"D:\BE\dl\archive\PlantVillage"
categories = os.listdir(dataset_path)
num_classes = len(categories)
image_size = 128
X = []
y = []


In [6]:
for category in categories:
    folder_path = os.path.join(dataset_path, category)
    label = categories.index(category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        image = cv2.imread(img_path)
        if image is not None:
            # Check if the image is not empty
            image = cv2.resize(image, (image_size, image_size))
            X.append(image)
            y.append(label)


In [7]:
# Convert lists to arrays
X = np.array(X)
y = to_categorical(y, num_classes)
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Create CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [9]:
# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
1032/1032 [==============================] - 234s 225ms/step - loss: 5.4925 - accuracy: 0.5160 - val_loss: 1.0592 - val_accuracy: 0.6731
Epoch 2/10
1032/1032 [==============================] - 286s 277ms/step - loss: 0.5956 - accuracy: 0.8208 - val_loss: 0.7197 - val_accuracy: 0.8020
Epoch 3/10
1032/1032 [==============================] - 438s 424ms/step - loss: 0.2360 - accuracy: 0.9289 - val_loss: 0.5358 - val_accuracy: 0.8803
Epoch 4/10
1032/1032 [==============================] - 316s 306ms/step - loss: 0.1506 - accuracy: 0.9558 - val_loss: 0.6023 - val_accuracy: 0.8699
Epoch 5/10
1032/1032 [==============================] - 256s 248ms/step - loss: 0.1373 - accuracy: 0.9619 - val_loss: 0.7780 - val_accuracy: 0.8446
Epoch 6/10
1032/1032 [==============================] - 214s 208ms/step - loss: 0.1239 - accuracy: 0.9657 - val_loss: 0.7823 - val_accuracy: 0.8819
Epoch 7/10
1032/1032 [==============================] - 203s 197ms/step - loss: 0.1309 - accuracy: 0.9623 - val_

In [17]:
# Save the trained model
model.save("plant_disease_model.h5")
from keras.models import load_model
# Load the saved model
model = load_model("plant_disease_model.h5")


In [18]:
# Provide the path to your custom input image
input_image_path = r"D:\BE\dl\paper.JPG"


In [19]:
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise Exception("Failed to read the image file.")
            image = cv2.resize(image, (image_size, image_size))
            image = image / 255.0 # Normalize the pixel values between 0 and 1
            return image
    except Exception as e:
        print("Error preprocessing image:", str(e))
        return None

In [20]:
# Function to make predictions on the input image
def predict_image(image_path):
    # Preprocess the input image
    image = preprocess_image(image_path)
    if image is not None:
        image = np.expand_dims(image, axis=0) # Add an extra dimension for batch size
        # Make the prediction
        prediction = model.predict(image)
        class_index = np.argmax(prediction)
        class_label = categories[class_index]
        return class_label
    else:
        return None

In [21]:
# Make a prediction on the input image
predicted_class = predict_image(input_image_path)
if predicted_class is not None:
    # Print the predicted class
    print("Predicted Class:", predicted_class)
